In [22]:
import json
import numpy as np
import pandas as pd

In [23]:
# get calinet
with open('../data/calinet_input_information.json', 'r') as f:
    data_calinet = json.load(f)
    calinet_df = pd.DataFrame(data_calinet).T

,fact_id,relation,subject,object,stem,true,false
0,1,P47,Norfolk,Suffolk,Norfolk shares border with,Suffolk,[Upper Macungie Township]
1,1,P47,Norfolk,Suffolk,Norfolk borders with,Suffolk,[Vadena]
2,1,P47,Norfolk,Suffolk,Norfolk shares a common border with,Suffolk,[Anabar]
3,1,P47,Norfolk,Suffolk,Suffolk borders with,Norfolk,[Riau]
4,1,P47,Norfolk,Suffolk,Suffolk shares its border with,Norfolk,[Bologna]


In [72]:
relations_used = []

In [73]:
stems_used = []

In [74]:
relations_used = list(calinet_df['relation'].values)

In [75]:
stems_used = list(calinet_df['stem'].values)

In [76]:
paper = ['calinet'] * len(stems_used)

In [77]:
assert(len(relations_used) == len(stems_used) == len(paper))

In [78]:
# get rome
with open('../data/rome_counterfact_original/counterfact.json', 'r') as f:
    data_rome = json.load(f)
    rome_df = pd.DataFrame(data_rome)

,case_id,pararel_idx,requested_rewrite,paraphrase_prompts,neighborhood_prompts,attribute_prompts,generation_prompts
0,0,2796,"{'prompt': 'The mother tongue of {} is', 'rela...",[Shayna does this and Yossel goes still and di...,"[The mother tongue of Léon Blum is, The native...","[J. R. R. Tolkien is a native speaker of, The ...","[Danielle Darrieux's mother tongue is, Where D..."
1,1,19501,"{'prompt': 'The official religion of {} is', '...",[The school chiefly served tribal girls of Dan...,"[The official religion of Charles Aznavour is,...",[Dave Chappelle is affiliated with the religio...,[Edwin of Northumbria's religious values stron...
2,2,13704,"{'prompt': '{}, the', 'relation_id': 'P1303', ...",[Initially and are zero and is false. Toko ...,"[Paul McCartney plays the instrument, John Len...","[Robert Radecke performs on the, Peter Igelhof...",[Toko Yasuda produces the most amazing music o...
3,3,6791,"{'prompt': '{}, which is located in', 'relatio...",[and Sallie Beavers Riley. Autonomous Universi...,"[Biure is located in, Ripollès, located in, Eb...","[SKF is located in, Köping Municipality, in, U...",[One can get to Autonomous University of Madri...
4,4,14712,{'prompt': 'What is the twin city of {}? It is...,"[Overall, however, Nápravník stayed true to Pu...","[What is the twin city of Los Angeles? It is, ...","[Sacramento is a twin city of, The twin city o...","[Lyon's twin city is known for, Lyon's twin ci..."


In [79]:
# get stems + relations for rome
for k in range(rome_df.shape[0]):
    paper.append('memit')
    stems_used.append(data_rome[k]['requested_rewrite']['prompt'].replace('{}', data_rome[k]['requested_rewrite']['subject']))
    relations_used.append(rome_df['requested_rewrite'][k]['relation_id'])

In [81]:
agg_df = pd.DataFrame({'relation': relations_used, 'stem': stems_used, 'paper': paper})

In [91]:
relation_ids = list(set(agg_df['relation']))

In [89]:
# lookup wikidata vals
wikidata_df = pd.read_csv('../data/wikidata/wikidata-property-list.csv')

In [106]:
relation_titles = []
for rel_id in relation_ids:
    relation_titles.append(list(wikidata_df[wikidata_df['ID'] == int(rel_id[1:])]['Title'])[0])

In [118]:
relation_lookup = dict(zip(relation_ids, relation_titles))
relation_titles_full = []

for i in range(agg_df.shape[0]):
    relation_titles_full.append(relation_lookup[agg_df['relation'][i]])

assert(len(relation_titles_full) == agg_df.shape[0])

In [120]:
agg_df['relation_title'] = relation_titles_full

In [121]:
agg_df.head()

,relation,stem,paper,relation_title
0,P47,Norfolk shares border with,calinet,shares border with
1,P47,Norfolk borders with,calinet,shares border with
2,P47,Norfolk shares a common border with,calinet,shares border with
3,P47,Suffolk borders with,calinet,shares border with
4,P47,Suffolk shares its border with,calinet,shares border with


In [129]:
agg_df.groupby(["relation", "relation_title"]).size()

relation  relation_title                             
P101      field of work                                  3824
P103      native language                                 919
P106      occupation                                     2318
P108      employer                                        350
P127      owned by                                        433
P1303     instrument                                      513
P131      is in the administrative territorial entity     714
P136      genre                                          1814
P1376     capital of                                     3509
P138      named after                                    4008
P140      religion                                        430
P1412     languages spoken                                924
P159      headquarters location                          2485
P17       country                                         875
P176      manufacturer                                   2188
P178      develo

In [133]:
agg_df[agg_df['paper'] == 'memit'][:15]

,relation,stem,paper,relation_title
50451,P103,The mother tongue of Danielle Darrieux is,memit,native language
50452,P140,The official religion of Edwin of Northumbria is,memit,religion
50453,P1303,"Toko Yasuda, the",memit,instrument
50454,P17,"Autonomous University of Madrid, which is loca...",memit,country
50455,P190,What is the twin city of Lyon? It is,memit,sister city
50456,P103,The mother tongue of Thomas Joannes Stieltjes is,memit,native language
50457,P740,"Anaal Nathrakh, that was created in",memit,formation location
50458,P178,Apple A5 was created by,memit,developer
50459,P190,What is the twin city of Wellington? It is,memit,sister city
50460,P495,"Shree Pundalik, created in",memit,country of origin
